# Analysis of Ontario Death Rates
> Investigating the Covid-19 deaths across the province.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/deaths.png

In [48]:
#hide
# import urllib, json, os, time
# import dateutil.parser
# import re

import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
import altair as alt

from IPython.display import HTML, display
import pytz

import ipywidgets as widgets

from datetime import datetime, timedelta, timezone

# import geopandas as gpd
# import sweetviz as sv

print('Libraries successfully imported.')

Libraries successfully imported.


In [49]:
#hide
#%%time

url = 'https://data.ontario.ca/dataset/f4112442-bdc8-45d2-be3c-12efae72fb27/resource/455fd63b-603d-4608-8216-7d8647f43350/download/conposcovidloc.csv'
url_2 = 'https://data.ontario.ca/dataset/f4f86e54-872d-43f8-8a86-3892fd3cb5e6/resource/ed270bb8-340b-41f9-a7c6-e8ef587e6d11/download/covidtesting.csv'
url_3 = 'https://data.ontario.ca/dataset/f4f86e54-872d-43f8-8a86-3892fd3cb5e6/resource/8a88fe6d-d8fb-41a3-9d04-f0550a44999f/download/daily_change_in_cases_by_phu.csv'

#geo = gpd.read_file('https://data.ontario.ca/dataset/f4112442-bdc8-45d2-be3c-12efae72fb27/resource/4f39b02b-47fe-4e66-95b6-e6da879c6910/download/conposcovidloc.geojson') 

src_conpos = pd.read_csv(url, index_col=0, parse_dates=['Accurate_Episode_Date', 'Case_Reported_Date', 'Test_Reported_Date', 'Specimen_Date']).reset_index()
src_testing = pd.read_csv(url_2, index_col=0, parse_dates=['Reported Date']).reset_index()
src_daily = pd.read_csv(url_3, index_col=0, parse_dates=['Date']).reset_index()

# duplicate entry for December 5th, 2020 causing error
src_testing.drop_duplicates(inplace=True)

# one row has date 3009-12-29
src_conpos[src_conpos['Accurate_Episode_Date'] == '3009-12-29'].replace('3009-12-29', '2020-11-29')
src_conpos['Accurate_Episode_Date'] = pd.to_datetime(src_conpos['Accurate_Episode_Date'], errors='coerce')

refreshtime = datetime.now()
cached=False

print('Source files successfully loaded.')

Source files successfully loaded.


In [50]:
#hide
last_date = src_testing['Reported Date'].max()
init_date = src_conpos['Accurate_Episode_Date'].min()
display(HTML("Ontario data set last updated on: " + last_date.strftime("%x")))

In [51]:
#hide_input
src_testing[src_testing['Confirmed Negative'] > 0].tail() # no confirmed negatives after March 29th 2020

testing_latest = src_testing[src_testing['Reported Date'] == src_testing['Reported Date'].max()]

# get the previous day results
testing_delta1 = src_testing[src_testing['Reported Date'] == src_testing['Reported Date'].max() - timedelta(days=1)]

# get the daily percentage +/-
dailypct = src_testing.join(src_testing[['Total Cases', 'Resolved', 'Confirmed Positive', 'Deaths']].pct_change().add_suffix('_pct'))

dailypct = dailypct[dailypct['Reported Date'] == dailypct['Reported Date'].max()]

# apply a percentage format; conventional formatting method throws an error
def percentage_delta(raw):
  # pctfmt = raw.map('{:+.2%}'.format)
  # split = str(pctfmt).split()
  formatted_pct = str(raw.map('{:+.2%}'.format)).split()[1]
  return formatted_pct

# original background colour code: #504e4e, secondary: #585858

display(HTML(
    
    "<div id='number-plate' style = 'background-color: #ececec; padding: 30px; text-align: center; marginTop: 1%; marginBottom: 1%;'>" +

    "<p style='color: black; font-size: 32px'> Ontario Covid-19 Cases to Date</p>"
    
    "<div id='confirmed' style = 'display: inline-block'> <span style='color: black; font-size: 23px;'> Confirmed: " +
    str('{:,.0f}'.format(int(testing_latest['Total Cases']))) + "</span>" + 
    "<br> <span style='color: black; font-size:15px; '> " + 
    str('{0:+,d}'.format(int(testing_latest['Total Cases']) - (int(testing_delta1['Total Cases'])))) +
    ' (' + percentage_delta(dailypct['Total Cases_pct']) + ')' +
    "</span>" + 
    "</div>" +

    "<div id='resolved' style = 'display: inline-block'> <span style='color: #228b22; font-size: 23px; margin-left: 20px;'> Resolved: " + 
    str('{:,.0f}'.format(int(testing_latest['Resolved']))) + "</span>" + 
    "<br> <span style='color: black; font-size:15px; '>" + 
    str('{0:+,d}'.format(int(testing_latest['Resolved']) - (int(testing_delta1['Resolved'])))) +
    ' (' + percentage_delta(dailypct['Resolved_pct']) + ')' +
    "</span>" + 
    "</div>" +

    "<div id='active' style = 'display: inline-block'> <span style='color: orange; font-size: 23px; margin-left: 20px;'> Active: " + 
    str('{:,.0f}'.format(int(testing_latest['Confirmed Positive']))) +"</span>" + 
    "<br> <span style='color: black; font-size:15px; '>" + 
    str('{0:+,d}'.format(int(testing_latest['Confirmed Positive']) - (int(testing_delta1['Confirmed Positive'])))) +
    ' (' + percentage_delta(dailypct['Confirmed Positive_pct']) + ')' +
    "</span>" + 
    "</div>" +

    "<div id='deaths' style = 'display: inline-block'> <span style='color: red; font-size: 23px; margin-left: 20px;'> Deaths: " + 
    str('{:,.0f}'.format(int(testing_latest['Deaths']))) + "</span>" + 
    "<br> <span style='color: black; font-size:15px; '>" + 
    str('{0:+,d}'.format(int(testing_latest['Deaths']) - (int(testing_delta1['Deaths'])))) +
    ' (' + percentage_delta(dailypct['Deaths_pct']) + ')' +
    "</span>" + 
    "</div>" +

    "<br><div id='rates' style = 'display: block; marginTop: 1%;'> " +

    "<div id='deathrate' style = 'display: inline-block'> <span style='color: black;'> Current Death Rate: " +
    "</span>" +
    "<span style = 'color: red'>" +
    str((testing_latest['Deaths'] / testing_latest['Total Cases']).map('{:.2%}'.format)).split()[1] + 
    "</span>" +
    "</div>" +

    "<div id='survivalrate' style = 'display:inline-block'> <span style='color: black; margin-left: 20px;'> Current Survival Rate: " + 
    "</span>" +
    "<span style = 'color: #228b22'>" +
    str(((testing_latest['Total Cases'] - testing_latest['Deaths']) / testing_latest['Total Cases']).map('{:.2%}'.format)).split()[1] + 
    "</span>" +
    "</div>" +

    "</div>" +

    "<span style='color: black; font-size: 11px;'> Ontario data last updated: " + 
    str(testing_latest['Reported Date'].max().strftime('%Y-%m-%d')) + ", Refreshed on: " + datetime.now(pytz.timezone('America/New_York') ).strftime("%Y-%m-%d at %H:%M") +
    "</span>" +

    "</div>"
    
    ))

## Overview of Fatalities by Age Group

In [52]:
#hide_input

df_deaths = src_conpos[src_conpos['Outcome1'] == 'Fatal'].groupby(['Age_Group']).count()["Row_ID"] \
.reset_index().iloc[:,[0,1]].rename(columns={'Row_ID': 'Deaths'})

df_res = src_conpos[src_conpos['Outcome1'] == 'Resolved'].groupby(['Age_Group']).count()["Row_ID"] \
.reset_index().iloc[:,[0,1]].rename(columns={'Row_ID': 'Resolved'})

df_nres = src_conpos[src_conpos['Outcome1'] == 'Not Resolved'].groupby(['Age_Group']).count()["Row_ID"] \
.reset_index().iloc[:,[0,1]].rename(columns={'Row_ID': 'Not Resolved'})

# merge dataframes together and clean up

df_age = df_deaths.merge(df_res, on='Age_Group').merge(df_nres, on='Age_Group').replace('<20', '0-19') \
.rename(columns={'Age_Group': 'Age Group'}).sort_values('Age Group')

del [df_deaths, df_res, df_nres]

# include percentages
df_age['Death Rate'] = (df_age['Deaths'] / (df_age['Resolved'] + df_age['Not Resolved']))
df_age['Survival Rate'] = (1 - df_age['Death Rate'])

x = df_age \
.drop(columns=['Resolved', 'Not Resolved']) \
.style.format({"Death Rate": "{:.2%}",
                     "Survival Rate": "{:.2%}"}) \
                     .background_gradient(cmap='Reds', subset=['Deaths', 'Death Rate']) \
                     .background_gradient(cmap='Greens', subset=['Survival Rate']) \
                     .hide_index()

x.set_properties(**{'text-align': 'center'}).hide_index()

Age Group,Deaths,Death Rate,Survival Rate
0-19,1,0.00%,100.00%
20s,6,0.02%,99.98%
30s,11,0.04%,99.96%
40s,33,0.14%,99.86%
50s,137,0.58%,99.42%
60s,365,2.47%,97.53%
70s,753,10.15%,89.85%
80s,1547,26.01%,73.99%
90s,1376,40.35%,59.65%


In [53]:
#hide_input

# df_deaths = src_conpos[src_conpos['Outcome1'] == 'Fatal'].groupby(['Case_Reported_Date', 'Age_Group']).count()['Row_ID'] \
# .reset_index().rename(columns={"Row_ID": "Cases"})

# fig_deaths = px.bar(df_deaths,
#                     y='Cases',
#                     x='Case_Reported_Date',
#                     color='Age_Group',
#                     labels={"Case_Reported_Date" : "Reported Date", "Age_Group" : "Age Group", "Cases" : "Deaths"},
#                     color_discrete_sequence=px.colors.qualitative.Set2,
#                     title = "Ontario Covid-19 Deaths by Age Group"
# )

# fig_deaths.show()

## Long Term Care Homes

In [54]:
#hide
df_ltc = src_testing[['Reported Date','Deaths','Total Cases','Number of patients hospitalized with COVID-19','Number of patients in ICU with COVID-19','Number of patients in ICU on a ventilator with COVID-19','Total Positive LTC Resident Cases','Total LTC Resident Deaths','Total LTC HCW Deaths']] \
.fillna(0)

In [55]:
#hide
ltc = pd.read_csv('https://data.ontario.ca/dataset/42df36df-04a0-43a9-8ad4-fac5e0e22244/resource/0f8b343e-fc28-4ca5-9aab-c3a1d2c919f1/download/ltccovidsummary.csv', parse_dates=True) \
.rename(columns={'ï»¿Report_Data_Extracted': 'Reported_Date'}).reset_index(drop=True)
ltc

,Reported_Date,LTC_Homes_with_Active_Outbreak,LTC_Homes_with_Resolved_Outbreak,Confirmed_Active_LTC_Resident_Cases,Confirmed_Active_LTC_HCW_Cases,Total_LTC_Resident_Deaths,Total_LTC_HCW_Deaths
0,2020-04-24,145,31,2455.0,1120.0,625.0,1.0
1,2020-04-25,147,31,2520.0,1161.0,654.0,1.0
2,2020-04-26,150,32,2523.0,1187.0,671.0,1.0
3,2020-04-27,154,34,2491.0,1205.0,705.0,1.0
4,2020-04-28,159,34,2632.0,1361.0,775.0,1.0
...,...,...,...,...,...,...,...
238,2020-12-18,145,290,795.0,902.0,2481.0,8.0
239,2020-12-19,154,284,931.0,900.0,2502.0,8.0
240,2020-12-20,160,280,965.0,959.0,2508.0,8.0
241,2020-12-21,159,285,963.0,972.0,2537.0,8.0


In [56]:
#hide
df_ltc['LTC Resident Death Percentage'] = (df_ltc['Total LTC Resident Deaths'] / df_ltc['Deaths']).fillna(0)
df_ltc

,Reported Date,Deaths,Total Cases,Number of patients hospitalized with COVID-19,Number of patients in ICU with COVID-19,Number of patients in ICU on a ventilator with COVID-19,Total Positive LTC Resident Cases,Total LTC Resident Deaths,Total LTC HCW Deaths,LTC Resident Death Percentage
0,2020-01-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,2020-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,2020-01-28,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,2020-01-30,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,2020-01-31,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
317,2020-12-19,4125.0,153614.0,895.0,256.0,146.0,9755.0,2546.0,8.0,0.617212
318,2020-12-20,4150.0,155930.0,875.0,261.0,156.0,9845.0,2564.0,8.0,0.617831
319,2020-12-21,4167.0,158053.0,915.0,265.0,152.0,9938.0,2581.0,8.0,0.619390
320,2020-12-22,4188.0,160255.0,1005.0,273.0,172.0,10056.0,2592.0,8.0,0.618911


In [57]:
fig_ltc = px.bar(df_ltc,
                 x='Reported Date',
                 y='Deaths',
                 color='LTC Resident Death Percentage', range_color=[0,5]
                    
)

fig_ltc.add_trace(go.Scatter(
    x=df_ltc['Reported Date'],
    y=df_ltc['Deaths'],
    mode='lines',
    #name='7d Avg',
    line=dict(color="#057D41", dash="solid")
    ))

fig_ltc.show()